Lab 9: Network problems
=======================

**Author:** Dimitris Floros



## Install libraries & external tools



In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

## Import packages



Load the required packages for this notebook.



In [302]:
import pandas as pd
import numpy as np
import sys
import matplotlib as plt
from pyomo.environ import *

## Solve the factory-warehouses transportation problem in Pyomo



### Define the data



In [303]:
# Define the name of each item in the sets
set_P  = ['P1', 'P2', 'P3']
set_W  = ['W1', 'W2', 'W3']

# Define the data (dictionaries)
data_C = {
  ('P1','W1'):10,('P1','W2'):5,('P1','W3'):9,
  ('P2','W1'):4 ,('P2','W2'):7,('P2','W3'):6,
  ('P3','W1'):3 ,('P3','W2'):5,('P3','W3'):9
}

data_d = { 'W1': 400, 'W2': 600, 'W3': 300  }

data_s = { 'P1': 800, 'P2': 300, 'P3': 200  }

### Defining model, sets, parameters and decision variables



In [304]:
#Initialize Model
model=ConcreteModel()             # initialize the model

#Defining Sets
model.P=Set(initialize=set_P)     # set of plants
model.W=Set(initialize=set_W)     # set of warehouse

#Defining Parameters
model.C=Param(model.P,model.W,initialize=data_C)   #
model.d=Param(model.W,initialize=data_d)           #
model.s=Param(model.P,initialize=data_s)           #

#Defining Decision Variables
model.X=Var(model.P, model.W, domain=NonNegativeReals)   # #units produced inside

### Defining objective function



In [305]:
def obj_cost(model):
    cost_shipping = sum( model.C[p,w]*model.X[p,w] for p in model.P for w in model.W )
    return cost_shipping

model.cost=Objective(sense=minimize,rule=obj_cost)

### Defining constraints



In [306]:
def meet_demand(model,w):
  return ( sum(model.X[p,w] for p in model.P) ) == model.d[w]
model.meetdem=Constraint(model.W,rule=meet_demand)

#adding supply availability constraint
def avail_supply(model,p):
  return sum(model.X[p,w] for w in model.W) <= model.s[p]
model.availsupply=Constraint(model.P,rule=avail_supply)

### Solve the model



In [ ]:
opt=SolverFactory('glpk')
results = opt.solve(model)

if results.solver.status == 'ok':
    print( f"Cost = ${model.cost():6,.2f}" )
    print()
    print("Shipping quantities between plants and warehouses:")
    for p in model.P:
        for w in model.W:
            print( "X[{},{}] = {:6.1f} \t ".format(
                p, w, model.X[p,w].value
            ), end = " " )
        print()

## Constraints on the routes



Imagine that the trucking company that serves the route from plant
   1 to warehouse 2 can handle only 250 units. How to add this
   constraint?



### Define the data



In [308]:
# Define the name of each item in the sets
set_P  = ['P1', 'P2', 'P3']
set_W  = ['W1', 'W2', 'W3']

# Define the data (dictionaries)
data_C = {
  ('P1','W1'):10,('P1','W2'):5,('P1','W3'):9,
  ('P2','W1'):4 ,('P2','W2'):7,('P2','W3'):6,
  ('P3','W1'):3 ,('P3','W2'):5,('P3','W3'):9
}

data_d = { 'W1': 400, 'W2': 600, 'W3': 300  }

data_s = { 'P1': 800, 'P2': 300, 'P3': 200  }

### Defining model, sets, parameters and decision variables



In [309]:
#Initialize Model
model=ConcreteModel()             # initialize the model

#Defining Sets
model.P=Set(initialize=set_P)     # set of plants
model.W=Set(initialize=set_W)     # set of warehouse

#Defining Parameters
model.C=Param(model.P,model.W,initialize=data_C)   #
model.d=Param(model.W,initialize=data_d)           #
model.s=Param(model.P,initialize=data_s)           #

#Defining Decision Variables
model.X=Var(model.P, model.W, domain=NonNegativeReals)   # #units produced inside

### Defining objective function



In [310]:
def obj_cost(model):
    cost_shipping = sum( model.C[p,w]*model.X[p,w] for p in model.P for w in model.W )
    return cost_shipping

model.cost=Objective(sense=minimize,rule=obj_cost)

### Defining constraints



In [311]:
def meet_demand(model,w):
  return ( sum(model.X[p,w] for p in model.P) ) == model.d[w]
model.meetdem=Constraint(model.W,rule=meet_demand)

#adding supply availability constraint
def avail_supply(model,p):
  return sum(model.X[p,w] for w in model.W) <= model.s[p]
model.availsupply=Constraint(model.P,rule=avail_supply)

#constraint on supply ilnes
def supply_lines(model):
  return model.X['P1','W2'] <= 250
model.supplylines=Constraint(rule=supply_lines)

### Solve the model



In [ ]:
opt=SolverFactory('glpk')
results = opt.solve(model)

if results.solver.status == 'ok':
    print( f"Cost = ${model.cost():6,.2f}" )
    print()
    print("Shipping quantities between plants and warehouses:")
    for p in model.P:
        for w in model.W:
            print( "X[{},{}] = {:6.1f} \t ".format(
                p, w, model.X[p,w].value
            ), end = " " )
        print()

## Hiring contractors to meet demand



Consider that the demand at the 3 warehouses increases. The company
   now needs to buy from a contractor to meet demand. There are 3
   different contractors with different production and shipping costs.
   How to model the contractors?



### Define the data



In [313]:
# Define the name of each item in the sets
set_P  = ['P1', 'P2', 'P3']
set_W  = ['W1', 'W2', 'W3']
set_O  = ['C1', 'C2', 'C3']

# Define the data (dictionaries)
data_C = {
  ('P1','W1'):10,('P1','W2'):5,('P1','W3'):9,
  ('P2','W1'):4 ,('P2','W2'):7,('P2','W3'):6,
  ('P3','W1'):3 ,('P3','W2'):5,('P3','W3'):9
}

data_d = { 'W1': 600, 'W2': 850, 'W3': 550  }

data_s = { 'P1': 800, 'P2': 300, 'P3': 200  }

data_n = { 'C1': 240, 'C2': 238, 'C3': 237.5 }

data_K = {
  ('C1','W1'):3 ,('C1','W2'):6, ('C1','W3'):15,
  ('C2','W1'):11,('C2','W2'):4, ('C2','W3'):8,
  ('C3','W1'):7 ,('C3','W2'):9, ('C3','W3'):11
}

### Defining model, sets, parameters and decision variables



In [314]:
#Initialize Model
model=ConcreteModel()             # initialize the model

#Defining Sets
model.P=Set(initialize=set_P)     # set of plants
model.W=Set(initialize=set_W)     # set of warehouse
model.O=Set(initialize=set_O)     # set of contractors

#Defining Parameters
model.C=Param(model.P,model.W,initialize=data_C)   #
model.d=Param(model.W,initialize=data_d)           #
model.s=Param(model.P,initialize=data_s)           #
model.n=Param(model.O,initialize=data_n)           #
model.K=Param(model.O,model.W,initialize=data_K)   #

#Defining Decision Variables
model.X=Var(model.P, model.W, domain=NonNegativeReals)   # #units produced inside
model.Y=Var(model.O, model.W, domain=NonNegativeReals)   # #units produced outside

### Defining objective function



In [315]:
def obj_cost(model):
    cost_shipping = sum( model.C[p,w]*model.X[p,w] for p in model.P for w in model.W )
    cost_contractors = sum( (model.n[c] + model.K[c,w])*model.Y[c,w] for c in model.O for w in model.W )
    return cost_shipping + cost_contractors

model.cost=Objective(sense=minimize,rule=obj_cost)

### Defining constraints



In [316]:
def meet_demand(model,w):
  return ( sum(model.X[p,w] for p in model.P) + sum(model.Y[c,w] for c in model.O) ) == model.d[w]
model.meetdem=Constraint(model.W,rule=meet_demand)

#adding supply availability constraint
def avail_supply(model,p):
  return sum(model.X[p,w] for w in model.W) <= model.s[p]
model.availsupply=Constraint(model.P,rule=avail_supply)

#constraint on supply ilnes
def supply_lines(model):
  return model.X['P1','W2'] <= 250
model.supplylines=Constraint(rule=supply_lines)

### Solve the model



In [ ]:
opt=SolverFactory('glpk')
results = opt.solve(model)

if results.solver.status == 'ok':
    print( f"Cost = ${model.cost():6,.2f}" )
    print()
    print("Shipping quantities between plants and warehouses:")
    for p in model.P:
        for w in model.W:
            print( "X[{},{}] = {:6.1f} \t ".format(
                p, w, model.X[p,w].value
            ), end = " " )
        print()

    print()
    print("Contractor quantities shipped to warehouses:")
    for c in model.O:
        for w in model.W:
            print( "Y[{},{}] = {:6.1f} \t ".format(
                c, w, model.Y[c,w].value
            ), end = " " )
        print()

## What if we can only have one contractor?



Reformulate the problem to only allow one contractor to be selected.



### Define the data



In [318]:
# Define the name of each item in the sets
set_P  = ['P1', 'P2', 'P3']
set_W  = ['W1', 'W2', 'W3']
set_O  = ['C1', 'C2', 'C3']

# Define the data (dictionaries)
data_C = {
  ('P1','W1'):10,('P1','W2'):5,('P1','W3'):9,
  ('P2','W1'):4 ,('P2','W2'):7,('P2','W3'):6,
  ('P3','W1'):3 ,('P3','W2'):5,('P3','W3'):9
}

data_d = { 'W1': 600, 'W2': 850, 'W3': 550  }

data_s = { 'P1': 800, 'P2': 300, 'P3': 200  }

data_n = { 'C1': 240, 'C2': 238, 'C3': 237.5 }

data_K = {
  ('C1','W1'):3 ,('C1','W2'):6, ('C1','W3'):15,
  ('C2','W1'):11,('C2','W2'):4, ('C2','W3'):8,
  ('C3','W1'):7 ,('C3','W2'):9, ('C3','W3'):11
}

### Defining model, sets, parameters and decision variables



In [319]:
#Initialize Model
model=ConcreteModel()             # initialize the model

#Defining Sets
model.P=Set(initialize=set_P)     # set of plants
model.W=Set(initialize=set_W)     # set of warehouse
model.O=Set(initialize=set_O)     # set of contractors

#Defining Parameters
model.C=Param(model.P,model.W,initialize=data_C)   #
model.d=Param(model.W,initialize=data_d)           #
model.s=Param(model.P,initialize=data_s)           #
model.n=Param(model.O,initialize=data_n)           #
model.K=Param(model.O,model.W,initialize=data_K)   #

#Defining Decision Variables
model.X=Var(model.P, model.W, domain=NonNegativeReals)   # #units produced inside
model.Y=Var(model.O, model.W, domain=NonNegativeReals)   # #units produced outside
model.h=Var(model.O, domain=Binary)                      # 1 if contractor is selected, 0 otherwise

### Defining objective function



In [320]:
def obj_cost(model):
    cost_shipping = sum( model.C[p,w]*model.X[p,w] for p in model.P for w in model.W )
    cost_contractors = sum( (model.n[c] + model.K[c,w])*model.Y[c,w] for c in model.O for w in model.W )
    return cost_shipping + cost_contractors

model.cost=Objective(sense=minimize,rule=obj_cost)

### Defining constraints



In [321]:
def meet_demand(model,w):
  return ( sum(model.X[p,w] for p in model.P) + sum(model.Y[c,w] for c in model.O) ) == model.d[w]
model.meetdem=Constraint(model.W,rule=meet_demand)

#adding supply availability constraint
def avail_supply(model,p):
  return sum(model.X[p,w] for w in model.W) <= model.s[p]
model.availsupply=Constraint(model.P,rule=avail_supply)

#constraint on supply ilnes
def supply_lines(model):
  return model.X['P1','W2'] <= 250
model.supplylines=Constraint(rule=supply_lines)

#adding contractor binary constraint
def binary_contractor(model):
  return sum(model.h[c] for c in model.O) <= 1
model.bincon=Constraint(rule=binary_contractor)

#adding contractor quantity by binary
def contractor_quantity(model,c):
  # get the maximum number representable by floating point
  INF_NUM = sys.float_info.max
  return sum(model.Y[c,w] for w in model.W) <= INF_NUM * model.h[c]
model.conquant=Constraint(model.O, rule=contractor_quantity)

### Solve the model



In [ ]:
opt=SolverFactory('glpk')
results = opt.solve(model)

if results.solver.status == 'ok':
    print( f"Cost = ${model.cost():6,.2f}" )
    print()
    print("Shipping quantities between plants and warehouses:")
    for p in model.P:
        for w in model.W:
            print( "X[{},{}] = {:6.1f} \t ".format(
                p, w, model.X[p,w].value
            ), end = " " )
        print()

    print()
    print("Contractor quantities shipped to warehouses:")
    for c in model.O:
        for w in model.W:
            print( "Y[{},{}] = {:6.1f} \t ".format(
                c, w, model.Y[c,w].value
            ), end = " " )
        print()